In [1]:
sc.install_pypi_package('pandas==1.3.5')
sc.install_pypi_package('boto3')
sc.install_pypi_package('pyspark==2.3.4')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1687300297410_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…






    ERROR: Command errored out with exit status 1:
     command: /mnt/yarn/usercache/livy/appcache/application_1687300297410_0002/container_1687300297410_0002_01_000001/tmp/1687303293099-0/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/mnt/tmp/pip-install-vtm0utc7/pyspark/setup.py'"'"'; __file__='"'"'/mnt/tmp/pip-install-vtm0utc7/pyspark/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /mnt/tmp/pip-pip-egg-info-8a6pb3mr
         cwd: /mnt/tmp/pip-install-vtm0utc7/pyspark/
    Complete output (24 lines):
    Could not import pypandoc - required to package PySpark
    Couldn't find index page for 'pypandoc' (maybe misspelled?)
    No local packages or working download links found for pypandoc
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/mnt/tmp/pip-install-vtm0utc7/pys

In [11]:
import pandas
import re
import tempfile
import os
import gzip
import shutil
import subprocess
from pyspark.sql.functions import udf, col, lit
from pyspark.sql import functions as F

"""
Import Parquet As a DataFrame
"""

#Read in parquet file from public S3 bucket
parquet_s3 = "s3://macaquenaive/parquet/"
df_spark = spark.read.parquet(parquet_s3)

##Verify count 
df_spark.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'query'
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1687300297410_0002/container_1687300297410_0002_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 1990, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name)
AttributeError: 'DataFrame' object has no attribute 'query'



In [19]:
class Query():
    
    '''An example query class to hold query parameters'''
    
    def __init__(self,q_name,length='',v_call_top="",d_call_top="",j_call_top="",regex="",animal_id="",v_mutation=''):
        self.query_name = q_name
        self.v_call_top = v_call_top
        self.j_call_top = j_call_top
        self.d_call_top = d_call_top
        self.animal_id = animal_id
        self.v_mutation = v_mutation
        
        if not length:
            raise Exception("Length must be supplied")
        self.length = length
        self.regular_expression = regex
    
    
    
    def apply(self,df):
        
        '''Apply function will take in spark dataframe and apply query parameters to it if they exist
        
           Returns a filtered dataframe
        '''
        self.queried_dataframe = ""
        
        ##Lets get length
        ##self.queried_dataframe = df.filter(F.length(df.cdr3_aa) == self.length)
        
        self.queried_dataframe = df.filter(F.length(df.cdr3_aa) > self.length)
        
        ##If the rest of these were specified, add them to the filter
       
        if self.v_call_top:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.v_call_top == self.v_call_top)
     
        #if self.d_call_top:
            #self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.d_call_top == self.d_call_top)     
            
        if self.d_call_top:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.d_call_top.rlike(self.d_call_top))
                
        if self.j_call_top:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.j_call_top == self.j_call_top)       
        

        if self.regular_expression:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.cdr3_aa.rlike(self.regular_expression))
        
        if self.animal_id:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.animal_id == self.animal_id)
        
        if self.v_mutation:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.v_mutation < self.v_mutation)
            
        print("Found {} sequences".format(self.queried_dataframe.count()))
        return self.queried_dataframe
        
    
def write_out_hadoop_to_local(df, s3):
    '''Because spark is distributed, writing out results is done by each worker node. As a result, it breaks up the results into partions. 
    
    This function will search the hadoop file system and concatenate the parted file back to a single CSV file
    
    params df - the quried dataframe
    s3 - the local file path (make sure its chmod 777)
    '''
    
    #Output Name
    output_s3 = s3
    
    #A temporary file to write out Hadoop CSV
    t = tempfile.NamedTemporaryFile(delete=False)
    

    #Write out the CSV
    df.write.csv(t.name,mode='overwrite')
    with open(output_s3,'w') as f:
        f.write(",".join(df.columns)+'\n')
        
        print("Running HDFS concatenate\n{} > {}".format(' '.join(['hdfs','dfs','-cat','{}/*csv'.format(t.name)]),output_s3))
        subprocess.Popen(['hdfs','dfs','-cat','{}/*csv'.format(t.name)],stdout=f)
        

    
    print('Wrote CSV to {}'.format(output_s3))



#query is IGHD3-3/J6 of length 23 with an FGV motif at the 7th index position of the HCDR3
#my_query = Query('BG18_search.1',length=21)
#my_query = Query('BG18_search.2',d_call_top=r'IGHD3-41',length=21,v_mutation=0.03)
#my_query = Query('BG18_search.3',d_call_top=r'IGHD3-41',length=21,regex=r'TIFG')
#my_query_4_7 = Query('BG18_search.4.7',d_call_top=r'IGHD3-41',length=21,regex=r'^....TIFG')
#my_query_4_8 = Query('BG18_search.4.8',d_call_top=r'IGHD3-41',length=21,regex=r'^.....TIFG')
#my_query_4_9 = Query('BG18_search.4.9',d_call_top=r'IGHD3-41',length=21,regex=r'^......TIFG')
#my_query_5_7 = Query('BG18_search.5.7',d_call_top=r'IGHD3-41',length=21,regex=r'^....TIFG.....E')
#my_query_5_8 = Query('BG18_search.5.8',d_call_top=r'IGHD3-41',length=21,regex=r'^.....TIFG.....E')
#my_query_5_9 = Query('BG18_search.5.9',d_call_top=r'IGHD3-41',length=21,regex=r'^......TIFG.....E')
my_query = Query('BG18_search.6',d_call_top=r'IGHD3-41',length=21, regex=r'TIFG.....E',v_mutation=0.03)
#my_query = Query('BG18_search.7',d_call_top=r'IGHD3-41',length=1,v_mutation=0.03)

#To run query, pass the input object from above to apply
queried_df = my_query.apply(df_spark)
#queried4_7_df = my_query_5_7.apply(df_spark)
#queried4_8_df = my_query_5_8.apply(df_spark)
#queried4_9_df = my_query_5_9.apply(df_spark)

# #To get an output, we can write the matching antibodies as a CSV file to our local file system from the Hadoop File System 
# write_out_hadoop_to_local(queried_df,'/home/hadoop/output/mycsvoutput.csv')
# pct64_df = determine_freq_of_junction(r'.YDFWS.............V$','IGHJ6')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found 308 sequences

In [3]:
## This cell searches for IgM precursors and calculate the IgM precursor frequencies of all animals with c_call="IGHM*03".
## However, for dataset of animals RPz18, RGp18, RPb18, and REt18, only IgMs were sorted and sequenced, and some of their c_call reads were incomplete and therefore not reliable.
## To address this issue, we used the next cell to analyze precursor frequencies of these four animals separately from this cell.

class Query():
    
    '''An example query class to hold query parameters'''
    
    def __init__(self,q_name,length='',d_call_top="",regex="",animal_id="",c_call="",file_name=""):
        self.query_name = q_name
        self.d_call_top = d_call_top
        self.animal_id = animal_id
        self.c_call = c_call
        self.file_name = file_name
        
        if not length:
            raise Exception("Length must be supplied")
        self.length = length
        self.regular_expression = regex
    
    
    
    def apply(self,df):
        
        '''Apply function will take in spark dataframe and apply query parameters to it if they exist
        
           Returns a filtered dataframe
        '''
        self.queried_dataframe = ""
        
        ##Lets get length
        
        self.queried_dataframe = df.filter(F.length(df.cdr3_aa) > self.length)
        
        ##If the rest of these were specified, add them to the filter
                   
        if self.d_call_top:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.d_call_top.rlike(self.d_call_top))
                
        if self.c_call:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.c_call == self.c_call)      

        if self.regular_expression:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.cdr3_aa.rlike(self.regular_expression))
        
        if self.animal_id:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.animal_id == self.animal_id)
                
        if self.file_name:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.file_name.rlike(self.file_name))
                
        print("Found {} sequences".format(self.queried_dataframe.count()))
        return self.queried_dataframe
        
    
def write_out_hadoop_to_local(df, s3):
    '''Because spark is distributed, writing out results is done by each worker node. As a result, it breaks up the results into partions. 
    
    This function will search the hadoop file system and concatenate the parted file back to a single CSV file
    
    params df - the quried dataframe
    s3 - the local file path (make sure its chmod 777)
    '''
    
    #Output Name
    output_s3 = s3
    
    #A temporary file to write out Hadoop CSV
    t = tempfile.NamedTemporaryFile(delete=False)
    

    #Write out the CSV
    df.write.csv(t.name,mode='overwrite')
    with open(output_s3,'w') as f:
        f.write(",".join(df.columns)+'\n')
        
        print("Running HDFS concatenate\n{} > {}".format(' '.join(['hdfs','dfs','-cat','{}/*csv'.format(t.name)]),output_s3))
        subprocess.Popen(['hdfs','dfs','-cat','{}/*csv'.format(t.name)],stdout=f)
        

    
    print('Wrote CSV to {}'.format(output_s3))
my_query_0 = Query('BG18_search.0',length=1,c_call="IGHM*03")
my_query_1 = Query('BG18_search.1',length=21,c_call="IGHM*03")
my_query_2 = Query('BG18_search.2',d_call_top=r'IGHD3-41',length=21,c_call="IGHM*03")
my_query_3 = Query('BG18_search.3',d_call_top=r'IGHD3-41',length=21,regex=r'IFG[VL]',c_call="IGHM*03")
my_query_4_7 = Query('BG18_search.4.7',d_call_top=r'IGHD3-41',length=21,regex=r'^.....IFG[VL]',c_call="IGHM*03")
my_query_4_8 = Query('BG18_search.4.8',d_call_top=r'IGHD3-41',length=21,regex=r'^......IFG[VL]',c_call="IGHM*03")
my_query_4_9 = Query('BG18_search.4.9',d_call_top=r'IGHD3-41',length=21,regex=r'^.......IFG[VL]',c_call="IGHM*03")
my_query_5_7 = Query('BG18_search.5.7',d_call_top=r'IGHD3-41',length=21,regex=r'^.....IFG[VL]....E',c_call="IGHM*03")
my_query_5_8 = Query('BG18_search.5.8',d_call_top=r'IGHD3-41',length=21,regex=r'^......IFG[VL]....E',c_call="IGHM*03")
my_query_5_9 = Query('BG18_search.5.9',d_call_top=r'IGHD3-41',length=21,regex=r'^.......IFG[VL]....E',c_call="IGHM*03")
my_query_6 = Query('BG18_search.6',d_call_top=r'IGHD3-41',length=21, regex=r'IFG[VL]....E',c_call="IGHM*03")
my_query_7 = Query('BG18_search.7',d_call_top=r'IGHD3-41',length=1,c_call="IGHM*03")

#To run query, pass the input object from above to apply

queried0_df = my_query_0.apply(df_spark)
queried1_df = my_query_1.apply(df_spark)
queried2_df = my_query_2.apply(df_spark)
queried3_df = my_query_3.apply(df_spark)
queried4_7_df = my_query_4_7.apply(df_spark)
queried4_8_df = my_query_4_8.apply(df_spark)
queried4_9_df = my_query_4_9.apply(df_spark)
queried5_7_df = my_query_5_7.apply(df_spark)
queried5_8_df = my_query_5_8.apply(df_spark)
queried5_9_df = my_query_5_9.apply(df_spark)
queried6_df = my_query_6.apply(df_spark)
queried7_df = my_query_7.apply(df_spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found 394 sequences

In [3]:
#This cell searches for precursors and calculate the precursor frequencies of RPz18, RGp18, RPb18, and REt18 only.
## The dataset of animals RPz18, RGp18, RPb18, and REt18, only IgMs were sorted and sequenced, and some of their c_call reads were incomplete and therefore not reliable.
## To address this issue, we used this cell to analyze precursor frequencies of these four animals separately from the previous cell.

class Query():
    
    '''An example query class to hold query parameters'''
    
    def __init__(self,q_name,length='',d_call_top="",regex="",animal_id="",c_call="",file_name=""):
        self.query_name = q_name
        self.d_call_top = d_call_top
        self.animal_id = animal_id
        self.c_call = c_call
        self.file_name = file_name
        
        if not length:
            raise Exception("Length must be supplied")
        self.length = length
        self.regular_expression = regex
    
    
    
    def apply(self,df):
        
        '''Apply function will take in spark dataframe and apply query parameters to it if they exist
        
           Returns a filtered dataframe
        '''
        self.queried_dataframe = ""
        
        ##Lets get length
        
        self.queried_dataframe = df.filter(F.length(df.cdr3_aa) > self.length)
        
        ##If the rest of these were specified, add them to the filter
                   
        if self.d_call_top:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.d_call_top.rlike(self.d_call_top))
                
        if self.c_call:
            self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.c_call == self.c_call)      

        if self.regular_expression:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.cdr3_aa.rlike(self.regular_expression))
        
        if self.animal_id:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.animal_id == self.animal_id)
                
        if self.file_name:
             self.queried_dataframe = self.queried_dataframe.filter(self.queried_dataframe.file_name.rlike(self.file_name))
                
        print("Found {} sequences".format(self.queried_dataframe.count()))
        return self.queried_dataframe
        
    
def write_out_hadoop_to_local(df, s3):
    '''Because spark is distributed, writing out results is done by each worker node. As a result, it breaks up the results into partions. 
    
    This function will search the hadoop file system and concatenate the parted file back to a single CSV file
    
    params df - the quried dataframe
    s3 - the local file path (make sure its chmod 777)
    '''
    
    #Output Name
    output_s3 = s3
    
    #A temporary file to write out Hadoop CSV
    t = tempfile.NamedTemporaryFile(delete=False)
    

    #Write out the CSV
    df.write.csv(t.name,mode='overwrite')
    with open(output_s3,'w') as f:
        f.write(",".join(df.columns)+'\n')
        
        print("Running HDFS concatenate\n{} > {}".format(' '.join(['hdfs','dfs','-cat','{}/*csv'.format(t.name)]),output_s3))
        subprocess.Popen(['hdfs','dfs','-cat','{}/*csv'.format(t.name)],stdout=f)
        

    
    print('Wrote CSV to {}'.format(output_s3))
my_query_0 = Query('BG18_search.0',length=1,file_name=r'_Functional')
my_query_1 = Query('BG18_search.1',length=21,file_name=r'_Functional')
my_query_2 = Query('BG18_search.2',d_call_top=r'IGHD3-41',length=21,file_name=r'_Functional')
my_query_3 = Query('BG18_search.3',d_call_top=r'IGHD3-41',length=21,regex=r'IFG[VL]',file_name=r'_Functional')
my_query_4_7 = Query('BG18_search.4.7',d_call_top=r'IGHD3-41',length=21,regex=r'^.....IFG[VL]',file_name=r'_Functional')
my_query_4_8 = Query('BG18_search.4.8',d_call_top=r'IGHD3-41',length=21,regex=r'^......IFG[VL]',file_name=r'_Functional')
my_query_4_9 = Query('BG18_search.4.9',d_call_top=r'IGHD3-41',length=21,regex=r'^.......IFG[VL]',file_name=r'_Functional')
my_query_5_7 = Query('BG18_search.5.7',d_call_top=r'IGHD3-41',length=21,regex=r'^.....IFG[VL]....E',file_name=r'_Functional')
my_query_5_8 = Query('BG18_search.5.8',d_call_top=r'IGHD3-41',length=21,regex=r'^......IFG[VL]....E',file_name=r'_Functional')
my_query_5_9 = Query('BG18_search.5.9',d_call_top=r'IGHD3-41',length=21,regex=r'^.......IFG[VL]....E',file_name=r'_Functional')
my_query_6 = Query('BG18_search.6',d_call_top=r'IGHD3-41',length=21, regex=r'IFG[VL]....E',file_name=r'_Functional')
my_query_7 = Query('BG18_search.7',d_call_top=r'IGHD3-41',length=1,file_name=r'_Functional')

#To run query, pass the input object from above to apply

queried0_df = my_query_0.apply(df_spark)
queried1_df = my_query_1.apply(df_spark)
queried2_df = my_query_2.apply(df_spark)
queried3_df = my_query_3.apply(df_spark)
queried4_7_df = my_query_4_7.apply(df_spark)
queried4_8_df = my_query_4_8.apply(df_spark)
queried4_9_df = my_query_4_9.apply(df_spark)
queried5_7_df = my_query_5_7.apply(df_spark)
queried5_8_df = my_query_5_8.apply(df_spark)
queried5_9_df = my_query_5_9.apply(df_spark)
queried6_df = my_query_6.apply(df_spark)
queried7_df = my_query_7.apply(df_spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found 394 sequences

In [ ]:
pandas0_df = queried0_df.select('sequence_id','animal_id').toPandas()
pandas1_df = queried1_df.select('sequence_id','animal_id').toPandas()
pandas2_df = queried2_df.select('sequence_id','animal_id').toPandas()
pandas3_df = queried3_df.select('sequence_id','animal_id').toPandas()
pandas47_df = queried4_7_df.select('sequence_id','animal_id').toPandas()
pandas48_df = queried4_8_df.select('sequence_id','animal_id').toPandas()
pandas49_df = queried4_9_df.select('sequence_id','animal_id').toPandas()
pandas57_df = queried5_7_df.select('sequence_id','animal_id').toPandas()
pandas58_df = queried5_8_df.select('sequence_id','animal_id').toPandas()
pandas59_df = queried5_9_df.select('sequence_id','animal_id').toPandas()
pandas6_df = queried6_df.select('sequence_id','animal_id').toPandas()
pandas7_df = queried7_df.select('sequence_id','animal_id').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pandas4_df = pandas.concat([pandas47_df,pandas48_df,pandas49_df])
pandas5_df = pandas.concat([pandas57_df,pandas58_df,pandas59_df])

In [12]:
#print out the number of IgM sequences from all animals  
counts = pandas0_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           count
animal_id       
zRh15          1
zRh03          1
A3             1
A4             1
Rh6            1
Rh2            1
Rh-A13003      1
F130           1
E09            1
D11            2
D19            2
Rh1            2
A1             3
A5             3
zRh14          4
E10            4
D15            7
zRh06          7
A2             8
E07           11
E08           11
E06           17
E04           19
zRh11         20
D20           21
D7            24
E05           45
E03           68
E01          107

In [ ]:
#print out the number of IgM sequences from all animals in search 1  
counts = pandas1_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

In [ ]:
#print out the number of IgM sequences from all animals in search 2  
counts = pandas2_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

In [ ]:
#print out the number of IgM sequences from all animals in search 3  
counts = pandas3_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

In [ ]:
#print out the number of IgM sequences from all animals in search 4 
counts = pandas4_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

In [ ]:
#print out the number of IgM sequences from all animals in search 5  
counts = pandas5_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

In [13]:
#print out the number of IgM sequences from all animals in search 6  
counts = pandas6_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           count
animal_id       
zRh15          1
zRh03          1
A3             1
A4             1
Rh6            1
Rh2            1
Rh-A13003      1
F130           1
E09            1
D11            2
D19            2
Rh1            2
A1             3
A5             3
zRh14          4
E10            4
D15            7
zRh06          7
A2             8
E07           11
E08           11
E06           17
E04           19
zRh11         20
D20           21
D7            24
E05           45
E03           68
E01          107

In [13]:
#print out the number of IgM sequences from all animals in search 7  
counts = pandas7_df.groupby('animal_id').count().rename({'sequence_id':'count'},axis=1).sort_values('count')
pandas.set_option('display.max_rows', None)  
print(counts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           count
animal_id       
zRh15          1
zRh03          1
A3             1
A4             1
Rh6            1
Rh2            1
Rh-A13003      1
F130           1
E09            1
D11            2
D19            2
Rh1            2
A1             3
A5             3
zRh14          4
E10            4
D15            7
zRh06          7
A2             8
E07           11
E08           11
E06           17
E04           19
zRh11         20
D20           21
D7            24
E05           45
E03           68
E01          107